# Tipsbet Analysis


In [1]:
from tips import Tips
import datetime as dt

## Load Data
The Tips module loads data all the data between the given start and end dates (both default to today if not given)

In [ ]:
start = dt.datetime.now() - dt.timedelta(days=365)
end = dt.datetime(2020, 4, 6) # - dt.timedelta(days=4)
t = Tips(start, end, verbosity=2)
t.df

Getting data for 2019-07-24 - 2020-04-09
Adding 2019-07-24
Adding 2019-07-25
Adding 2019-07-26
Adding 2019-07-27
Adding 2019-07-28
Adding 2019-07-29
Adding 2019-07-30
Adding 2019-07-31
Adding 2019-08-01
Adding 2019-08-02
Adding 2019-08-03
Adding 2019-08-04
Adding 2019-08-05
Adding 2019-08-06
Adding 2019-08-07
Adding 2019-08-08
Adding 2019-08-09
Adding 2019-08-10
Adding 2019-08-11
Adding 2019-08-12
Adding 2019-08-13
Adding 2019-08-14
Adding 2019-08-15
Adding 2019-08-16
Adding 2019-08-17
Adding 2019-08-18
Adding 2019-08-19
Adding 2019-08-20
Adding 2019-08-21
Adding 2019-08-22
Adding 2019-08-23
Adding 2019-08-24
Adding 2019-08-25
Adding 2019-08-26
Adding 2019-08-27
Adding 2019-08-28
Adding 2019-08-29
Adding 2019-08-30
Adding 2019-08-31
Adding 2019-09-01
Adding 2019-09-02
Adding 2019-09-03
Adding 2019-09-04
Adding 2019-09-05
Adding 2019-09-06
Adding 2019-09-07
Adding 2019-09-08
Adding 2019-09-09
Adding 2019-09-10
Adding 2019-09-11
Adding 2019-09-12
Adding 2019-09-13
Adding 2019-09-14
Addin

In [ ]:
t.df.Status.value_counts(normalize=True)